In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.interpolate import spline

import ROOT
from ROOT import gROOT, gSystem, TFile, TGraphAsymmErrors, TH1, TF1, TFitResultPtr
import rootnotes
import rootprint


In [2]:
%%rootprint
vegasPath=os.path.expandvars("$VEGAS")
gROOT.Reset()
gSystem.Load("libTreePlayer.so")
gSystem.Load("libPhysics.so")
gSystem.Load(vegasPath + "/common/lib/libSP24sharedLite.so")
gSystem.Load(vegasPath + "/resultsExtractor/lib/libStage6shared.so")
gSystem.AddIncludePath("-Wno-unused -Wno-shadow -Wno-unused-parameter")
gROOT.ProcessLine(".L " + vegasPath + "/common/include/VACommon.h")
gROOT.ProcessLine(".include " + vegasPath + "/common/include/")
gROOT.ProcessLine(".include " + vegasPath + "/resultsExtractor/include/")
gROOT.ProcessLine(".include " + vegasPath + "/cfitsio/include/")

In [3]:
def get_hist_points(h):
    """return E[TeV], flux[1/TeV*m^2*s], and flux_err as lists"""
    x, y, yerr = [], [], []
    for pt in range(1, h.GetNbinsX()+1):
        tmpX, tmpY = ROOT.Double(0), ROOT.Double(0)
        #h.GetPoint(pt, tmpX, tmpY)
        x.append(h.GetBinCenter(pt))
        y.append(h.GetBinContent(pt))
        yerr.append(h.GetBinError(pt))
        
    #print np.array(x), np.array(y), np.array(yerr)
    return np.power(10, np.array(x)), np.array(y), np.array(yerr)
# get_hist_points



In [4]:
home = os.path.expanduser("~")
filename = home + "/VERITAS/GC/results/SgrA_disp5t_v255_okay-obs-4tel_medium_both_s6.root"
s6F = TFile(filename, "read")
if not s6F.IsOpen():
    print "Failure!"
specAn = s6F.Get("Spectrum/VASpectrumAnl")
hMM = s6F.Get("Spectrum/hMigrationMatrix")
hFEH = s6F.Get("Spectrum/hFullExcessHist")
RBM = s6F.Get("RingBackgroundModelAnalysis")

specHist = specAn.GetSpectrumHist()
rebinned_specHist = specAn.GetRebinnedSpectrumHist()
specGraph = specAn.GetSpectrumGraph()


In [5]:
xlab = specHist.GetXaxis().GetTitle()
ylab = specHist.GetYaxis().GetTitle()
print xlab

E, flux, flux_err = get_hist_points(rebinned_specHist)
print E
print flux
print flux_err

print type(specGraph)

E  [ Log10(TeV) ]
[  2.81838293   4.46683592   7.07945784  11.22018454  17.7827941 ]
[  3.64033803e-09   1.05522735e-09   3.23616467e-10   1.14158245e-10
   2.72773488e-11]
[  3.34238831e-10   1.08817913e-10   4.51352671e-11   2.11965827e-11
   8.43257755e-12]
<class 'ROOT.TGraphAsymmErrors'>


In [6]:
#%%rootprint
tf1 = specGraph.GetFunction("fFitFunction")
fitnorm = tf1.GetParameter(0)
fitindex = tf1.GetParameter(1)
normenergy = tf1.GetParameter(2)

r = specGraph.Fit(tf1, "S") #TFitResultPtr
#for i in range(3):
#    print tf1.GetParameter(i)
    
cov = r.GetCovarianceMatrix() #TMatrixTSym<double>
cov.Print()

var_norm = cov(0, 0)
var_index = cov(1, 1)
cov_normindex = cov(0, 1) # == (1, 0)

print np.sqrt(var_norm)
print np.sqrt(var_index)
print cov_normindex

#covarr = cov.GetMatrixArray()


9.44156106362e-09
0.113234169281
-1.01431019844e-09


In [53]:
npoints     = specGraph.GetN()

x2, y2 = [], []
y_err = []
for i in range(npoints):
    tmpX, tmpY = ROOT.Double(0), ROOT.Double(0)
    specGraph.GetPoint(i, tmpX, tmpY)
    x2.append(tmpX)
    y2.append(tmpY)
    y_err.append((specGraph.GetErrorYlow(i), specGraph.GetErrorYhigh(i)))
    
x2 = np.array(x2)
y2 = np.array(y2)
y_err = np.array(y_err)

print x2
print y2
print y_err

[  0.           0.           2.80374901   4.44364272   7.04269909
  11.16192585  17.69046029]
[  0.00000000e+00   0.00000000e+00   3.64033803e-09   1.05522735e-09
   3.23616467e-10   1.14158245e-10   2.72773488e-11]
[[  0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00]
 [  3.34238831e-10   3.34238831e-10]
 [  1.08817913e-10   1.08817913e-10]
 [  4.51352671e-11   4.51352671e-11]
 [  2.11965827e-11   2.11965827e-11]
 [  8.43257755e-12   8.43257755e-12]]


In [7]:
%%rootprint
s6F.ls()
cov.Print()


TFile**		/home/mbuchove/VERITAS/GC/results/SgrA_disp5t_v255_okay-obs-4tel_medium_both_s6.root	
 TFile*		/home/mbuchove/VERITAS/GC/results/SgrA_disp5t_v255_okay-obs-4tel_medium_both_s6.root	
  TDirectoryFile*		Spectrum	Spectrum
   OBJ: TH2F	hMigrationMatrix	 : 0 at: 0x4b05b30
   OBJ: TH1F	Stage6Spectrum_FullExcess	 : 0 at: 0x4b00960
   KEY: VASpectrumAnl	VASpectrumAnl;1	Spectrum
   KEY: TH2F	hMigrationMatrix;1	
   KEY: TH1F	hFullExcessHist;1	
  TDirectoryFile*		RingBackgroundModelAnalysis	RingBackgroundModelAnalysis
   KEY: TDirectoryFile	ExclusionRegions;1	ExclusionRegions
   KEY: TDirectoryFile	AcceptanceCurves;1	AcceptanceCurves
   KEY: VASkyMap	SkyMapOn;1	
   KEY: VASkyMap	SkyMapOff;1	
   KEY: VASkyMap	SkyMapDiff;1	
   KEY: VASkyMap	SkyMapSig;1	
   KEY: VASkyMap	fRingBackground;1	
   KEY: VASkyMap	fAlphaMap;1	
   KEY: VASkyMap	fAcceptanceMap;1	
   KEY: VASkyMap	fIntAccMap;1	
   KEY: VASkyMap	fIntOnMap;1	
   KEY: VASkyMap	fSigMap;1	
   KEY: VASkyMap	fExcessMap;1	
   KEY: VASkyMap	fUn